In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import kagglehub
import pandas as pd
import re

In [ ]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Удаление пунктуации
    text = text.lower()  # Приведение к нижнему регистру
    return text

In [ ]:
path = kagglehub.dataset_download("vfomenko/russian-news-2020")
file_path = f"{path}/news.csv"
data = pd.read_csv(file_path)
print(data.head())

100%|██████████| 19.9M/19.9M [00:00<00:00, 81.3MB/s]

Extracting files...


     source                                           title  \
0  lenta.ru                                  Синий богатырь   
1  lenta.ru  Загитова согласилась вести «Ледниковый период»   
2  lenta.ru       Объяснена опасность однообразного питания   
3  lenta.ru                      «Предохраняться? А зачем?»   
4  lenta.ru     Ефремов систематически употреблял наркотики   

                                                text  \
0  В 1930-е годы Советский Союз охватила лихорадк...   
1  Олимпийская чемпионка по фигурному катанию  Ал...   
2  Российский врач-диетолог Римма Мойсенко объясн...   
3  В 2019 году телеканал «Ю» запустил адаптацию з...   
4  Актер  Михаил Ефремов  систематически употребл...   

            publication_date          rubric     subrubric tags  
0  2020-08-30T00:01:00+03:00       Экономика  Госэкономика  NaN  
1  2020-08-31T20:04:00+03:00           Спорт   Зимние виды  NaN  
2  2020-08-31T20:07:00+03:00        Из жизни           Еда  NaN  
3  2020-08-30T00:04:

In [ ]:
# Создание словаря символов
data['text'] = data['text'].fillna("")
data['text'] = data['text'].apply(clean_text)

# Создание словаря слов
words = " ".join(data['text']).split()
word_to_index = {word: i for i, word in enumerate(set(words))}
index_to_word = {i: word for i, word in enumerate(set(words))}

# Преобразование текста в последовательности слов
max_sequence_length = 10  # Максимальная длина последовательности
X = []
y = []

for text in data['text']:
    tokens = text.split()
    for i in range(len(tokens) - 1):
        input_seq = tokens[:i + 1]
        target_word = tokens[i + 1]
        X.append([word_to_index[word] for word in input_seq])
        y.append(word_to_index[target_word])

# Дополнение последовательностей до одинаковой длины
X = pad_sequences(X, maxlen=max_sequence_length, padding='pre')

# Преобразование целевых значений в one-hot encoding
y = to_categorical(y, num_classes=len(word_to_index))

In [ ]:
path = kagglehub.dataset_download("demartlectus/wildberries-search-queries-from-wildhack-2021")
file_path = f"{path}/query_popularity.csv"
data = pd.read_csv(file_path)
print(data.head())

                    query  query_popularity
0                 ноутбук                10
1  куртка женская осенняя                10
2         ботинки женские                10
3              видеокарта                10
4  пальто женское осеннее                10


In [ ]:
# Создание словаря символов
data['query'] = data['query'].fillna("")
data['query'] = data['query'].apply(clean_text)

# Создание словаря слов
words = " ".join(data['query']).split()
word_to_index = {word: i for i, word in enumerate(set(words))}
index_to_word = {i: word for i, word in enumerate(set(words))}

# Преобразование текста в последовательности слов
max_sequence_length = 10  # Максимальная длина последовательности
X = []
y = []

for text in data['query']:
    tokens = text.split()
    for i in range(len(tokens) - 1):
        input_seq = tokens[:i + 1]
        target_word = tokens[i + 1]
        X.append([word_to_index[word] for word in input_seq])
        y.append(word_to_index[target_word])

# Дополнение последовательностей до одинаковой длины
X = pad_sequences(X, maxlen=max_sequence_length, padding='pre')

# Преобразование целевых значений в one-hot encoding
y = to_categorical(y, num_classes=len(word_to_index))

In [ ]:
# Параметры модели
embedding_dim = 50
lstm_units = 128

# Создание модели
model = Sequential([
    Embedding(input_dim=len(words), output_dim=embedding_dim, input_length=max_sequence_length),
    LSTM(lstm_units),
    Dense(len(words), activation='softmax')
])

# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(X, y, epochs=50, batch_size=32)

NameError: name 'Sequential' is not defined

In [ ]:
# Функция для генерации текста
def generate_text(seed_text, num_chars_to_generate, temperature=1.0):
    for _ in range(num_chars_to_generate):
        # Преобразование seed_text в последовательность индексов
        input_seq = [char_to_index[char] for char in seed_text]
        input_seq = pad_sequences([input_seq], maxlen=max_sequence_length, padding='pre')

        # Предсказание вероятностей следующего символа
        predicted_probs = model.predict(input_seq, verbose=0)[0]

        # Применение температуры для "смягчения" вероятностей
        predicted_probs = np.log(predicted_probs) / temperature
        predicted_probs = np.exp(predicted_probs)
        predicted_probs = predicted_probs / np.sum(predicted_probs)

        # Случайный выбор символа на основе вероятностей
        predicted_index = np.random.choice(len(chars), p=predicted_probs)
        predicted_char = index_to_char[predicted_index]

        # Добавление предсказанного символа к seed_text
        seed_text += predicted_char

    return seed_text

# Генерация текста
seed_texts = ["шуба", "акриловые", "я"]
for seed in seed_texts:
    generated_text = generate_text(seed, num_chars_to_generate=50)
    print(f"Начальная фраза: '{seed}'")
    print(f"Сгенерированный текст: {generated_text}")
    print("-" * 50)

Начальная фраза: 'шуба'
Сгенерированный текст: шубаictooubucebcleitrinbbiluouarirtoibbuucuounibsa_ece
--------------------------------------------------
Начальная фраза: 'акриловые'
Сгенерированный текст: акриловыеtiiilabarcrctucucuauluiuiciittxcgcuuuaioelrtcnedao
--------------------------------------------------
Начальная фраза: 'я'
Сгенерированный текст: яr_buioairenebnbetbcliebaabiucclicictablrrurrbeаbtt
--------------------------------------------------
